# <a id='toc1_'></a>[How the sing_curr_conv works](#toc0_)

This document demonstrates how the **sing_curr_conv** (SCC)  converts the original ledger to the equivalent one in different scenarios. 
For the demonstration purposes this document is created in a form of the Jupyter note book, however the **sing_curr_conv**  can provide the same functionality also from the command line or as a plugin (refer to the [sing_curr_conv.md](sing_curr_conv.md) for more information). 

You can read this as a document and/or interactively experiment by changing parameters

**Table of contents**<a id='toc0_'></a>    
- [How the sing_curr_conv works](#toc1_)    
  - [Initial notebook setup](#toc1_1_)    
    - [Imports](#toc1_1_1_)    
    - [Some helper functions](#toc1_1_2_)    
  - [How the sing_curr_conv  works](#toc1_2_)    
    - [Simple case of price change](#toc1_2_1_)    
    - [Currency exchange at not market price](#toc1_2_2_)    
    - [Tracking at cost, selling with gain](#toc1_2_3_)    
    - [Multiple Balance Sheet accounts. Usage of the "group_p_l_acc_tr" option.](#toc1_2_4_)    
    - [Unconvertable commodity](#toc1_2_5_)    
      - [Usage of the "start_date" and "end_date" options](#toc1_2_5_1_)    
    - [Small value Balance error correction posting](#toc1_2_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Initial notebook setup](#toc0_)
This section contains some code, needed for initial notebook setup and is not related to the functionality of the SCC

### <a id='toc1_1_1_'></a>[Imports](#toc0_)

In [1]:
import textwrap
import traceback

from beancount.loader import load_string
from beancount.parser.printer import print_entries

from evbeantools.sing_curr_conv import get_equiv_sing_curr_entries, print_errors_to_string

from evbeantools.printer_rich import display_entries

### <a id='toc1_1_2_'></a>[Some helper functions](#toc0_)
Let us define some functions, which we will use during demonstration

In [2]:
def display_scc_converted(ledger:str, **kwargs):
    """
    Converts the leger to a single currency and returns the HTML rich-formatted text of the converted ledger, whapped to 
    IPython displayable format.
    Args:
        ledger (str): beancount ledger file.
        **kwargs: Keyword arguments to be passed to the get_equiv_sing_curr_entries function.
    """
    ledger = textwrap.dedent(ledger)
    
    entries, errors, options = load_string(ledger)
    
    if len(errors) > 0:
        raise RuntimeError(f"Errors in the original ledger file\n {print_errors_to_string(errors)}")
    
    try:
        entries_eqv, errors_eqv, options_eqv = get_equiv_sing_curr_entries(entries, options, **kwargs)
    except Exception as e:
        print(f"An exception caught, while running the 'get_equiv_sing_curr_entries':\n {e}")
        return
    
    if len(errors_eqv) > 0:
        raise RuntimeError("Errors in the converted ledger file")
    
    return(display_entries(entries_eqv))

def check_for_errors(ledger:str):
    """
    Loads the ledger and checks for errors. If there are errors, prints them to the console.
    If there are no errors, prints "No errors in the ledger file".
    Args:
        ledger (str): beancount leger file.
    """
    ledger = textwrap.dedent(ledger)
    
    entries, errors, options = load_string(ledger)
    
    if len(errors) > 0:
        print(f"There are errors!:\n{print_errors_to_string(errors)}")
    else:
        print("No errors in the ledger file")

## <a id='toc1_2_'></a>[How the sing_curr_conv  works](#toc0_)

### <a id='toc1_2_1_'></a>[Simple case of price change](#toc0_)

Let us explore the most basic case.
We have 100 of AAA commodity in the bank account, but we want to report everything in the BBB commodity.

On the 1 February 2020  the exchange rate of the AAA to BBB changes from 1.0 to 2.0 

In [3]:
ledger_simple_case = """

option "operating_currency" "BBB" 

2020-01-01 open Assets:Bank
2020-01-01 open Equity:Opening-Balances

2020-01-01 price AAA 1.0 BBB

2020-01-01 * "Opening Balance"
    Assets:Bank                100 AAA
    Equity:Opening-Balances   -100 AAA

; Note: at this moment out net worth, when measured in BBB, is 100 BBB, as the exchange rate is 1.0

2020-02-01 price AAA 2.0 BBB

; Note: at this moment out net worth, when measured in BBB, is 200 BBB, as the exchange rate has changed

"""

One can see, that if we want to report everything in BBB, then on the 1 February 2020 our net worth has changed from 100 BBB to 200 BBB.
But since this was caused by a price change and not a transaction, there is no *beanquery* query, which would explain us why our net worth has increased

Let is see, how the SCC handles this

In [4]:
display_scc_converted(ledger_simple_case)

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x217577c0950>

Note that SCC has performed the following actions:

- Converted all entries in the Balance Sheet accounts to **BBB** currency at the exchange rate applicable on the given date.  
- Added metadata with extra information  
- Created an unrealized gains transaction dated on the day of the price change.  

Let’s take a closer look at the unrealized gains transaction. It has two postings:

- One increases the balance in **Assets:Bank** by the amount of the increase resulting from the price change.  
- The other books the same amount (but with the opposite sign) to the newly created  
  **Income:Unrealized-Gains:\<Target Currency\>-\<Changed Currency\>** account. Where:  
  - **\<Target Currency\>** is the currency in which the unrealized gain is analyzed (BBB in our case).  
  - **\<Changed Currency\>** is the currency whose exchange rate changed relative to the target currency, causing the unrealized gain (AAA in our case).  

This unrealized gain transaction has one piece of transaction-level metadata (indicating how the transaction was created) and several posting-level metadata entries.

**Posting-Level Metadata**

- **scc_msg**  
  Explains what the Balance Sheet account balance was that led to this unrealized gain transaction. Remember that in Beancount, a new price takes effect at the start of the day. Therefore, this balance reflects the start of the day (i.e., the end of the previous day).  

- **scc_at_cost**  
  Indicates whether the position triggering this unrealized gain was tracked at cost. This can be useful for future analysis (e.g., positions tracked at cost often incur capital gains tax). In this case, the position was *not* tracked at cost.  

- **scc_unreal_g_cause**  
  States the reason for the unrealized gain transaction. Here, it is `price_change`. Later, we will see that other values are possible.  

- **scc_bal_s_acc**  
  This metadata is only present on the posting to the **Income:Unrealized-Gains** account and corresponds to the Balance Sheet account name of the other posting. It can be used to analyze which Balance Sheet accounts caused unrealized gains.  

With this new ledger, converted to the target currency, we have all the information we need to determine and analyze unrealized gains using **beanquery**.

### <a id='toc1_2_2_'></a>[Currency exchange at not market price](#toc0_)
Let us look at the situation, when there was a currency exchange at a price / exchange rate, different from the one specified by the price directive

In [5]:
ledger_exchange_wrong_price = """

option "operating_currency" "BBB" 

2020-01-01 commodity AAA
2020-01-01 commodity BBB

2020-01-01 open Assets:BankA              AAA
2020-01-01 open Assets:BankB              BBB
2020-01-01 open Equity:Opening-Balances   AAA

2020-01-01 price AAA 1.0 BBB

2020-01-01 * "Opening Balance"
    Assets:BankA                100 AAA
    Equity:Opening-Balances    -100 AAA

2020-02-01 * "Exchange AAA to BBB"
    Assets:BankA               -100 AAA
    Assets:BankB                150 BBB @@ 100 AAA ; <== price, different from the one in the price directive

"""

check_for_errors(ledger_exchange_wrong_price)

No errors in the ledger file


Beancount does allow this situation, and does not complain about it. Not even if you use a combination of [implicit_prices](https://github.com/beancount/beancount/blob/master/beancount/plugins/implicit_prices.py) and [pedantic](https://github.com/beancount/beancount/blob/master/beancount/plugins/pedantic.py) plugins

In [6]:
ledger_exchange_wrong_price_implicit_pedantic = """

option "operating_currency" "BBB" 

plugin "beancount.plugins.implicit_prices"

plugin "beancount.plugins.pedantic"


2020-01-01 commodity AAA
2020-01-01 commodity BBB

2020-01-01 open Assets:BankA              AAA
2020-01-01 open Assets:BankB              BBB
2020-01-01 open Equity:Opening-Balances   AAA

2020-01-01 price AAA 1.0 BBB

2020-01-01 * "Opening Balance"
    Assets:BankA                100.0 AAA
    Equity:Opening-Balances    -100.0 AAA

2020-02-01 * "Exchange AAA to BBB"
    Assets:BankA               -100.0 AAA
    Assets:BankB                150.0 BBB @@ 100 AAA ; <== price, different from the one in the price directive

"""

check_for_errors(ledger_exchange_wrong_price_implicit_pedantic)

No errors in the ledger file


Later however **beanquery** is going to use only the price directive prices for its CONVERT function. 

In general, when you buy something at a price significantly different than the market price, this shall be treated as a gain or loss. However as we see it is quite easy to miss such situation, when using a vanilla beancount. This is not to mention the fact, that in many countries the situation, when you buy something at a price, significantly different from the market price, may attract an attention of the authorities, when happening on a large scale. 

Let us check how the SCC handles this.

In [7]:
display_scc_converted(ledger_exchange_wrong_price)

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x21757b2a5d0>

Observe that for the **Exchange AAA to BBB** transaction, the SCC has converted the posting to the **Assets:BankA** account into the new commodity (**BBB**) at the exchange rate specified by the price directive and has added an unrealized gain posting to the **Income:Unrealized-Gains:BBB-AAA** account, which accounts for the gain resulting from the exchange rate differing from the one specified in the price directive. Also note that the **scc_unreal_g_cause** metadata now has the value **price_diff**, rather than **price_change**.


### <a id='toc1_2_3_'></a>[Tracking at cost, selling with gain](#toc0_)
Let us look at the situation, when certain commodity was purchased, tracked at cost and then sold with gain

In [8]:
ledger_tracking_at_cost_at_selling_with_gain = """

option "operating_currency" "USD" 

2020-01-01 open Assets:Bank:Checking 
2020-01-01 open Equity:Opening-Balances
2020-01-01 open Assets:Investments:IVV
2020-01-01 open Income:Investment

2020-01-01 * "Opening Balances"
    Assets:Bank:Checking  1000 USD
    Equity:Opening-Balances
    
2020-01-01 price IVV 150 USD
    
2020-01-02 * "Buying IVV at cost at a market price"
    Assets:Bank:Checking    -300 USD
    Assets:Investments:IVV     2 IVV {150 USD} 
    
    
2020-02-01 price IVV 200 USD


2020-03-01 * "Selling IVV at a new price"
    Assets:Bank:Checking        400 USD 
    Income:Investment          -100 USD
    Assets:Investments:IVV       -2 IVV {150 USD} @ 200 USD 
"""

display_scc_converted(ledger_tracking_at_cost_at_selling_with_gain)

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x217580bc170>

Observe, that there are 2 posting to the account **Income:Unrealized-Gains:USD-IVV**

-  the first one on the 2020-02-01, and it is caused by market price change
-  the second one is on the 2020-03-01, when the commodity IVV was sold with the gain. This posting cancels out the previous one, as it has the same value, but an opposite sign. This is correct, as the gain, which existed in the "unrealized form" before 2020-03-01 has now been realized and booked to the **Income:Investment** account. 
  
Both postings have the value of the meta **scc_at_cost** equal to *at_cost*, which is different from the previous example, where this meta was equal to the "no_cost". Filtering or grouping by in beanquery by the transaction - level metadata **scc_at_cost** allows to separate unrealized gains for the positions, tracked at cost, and positions not tracked at cost.


### <a id='toc1_2_4_'></a>[Multiple Balance Sheet accounts. Usage of the "group_p_l_acc_tr" option.](#toc0_)
Let us look at the situation, when there are several Balance Sheet accounts with a commodity, which price is changing

In [9]:
ledger_simple_case_mult_acc = """

option "operating_currency" "BBB" 

2020-01-01 open Assets:Bank1
2020-01-01 open Assets:Bank2
2020-01-01 open Equity:Opening-Balances

2020-01-01 price AAA 1.0 BBB

2020-01-01 * "Opening Balance"
    Assets:Bank1                100 AAA
    Assets:Bank2                100 AAA
    Equity:Opening-Balances   - 200 AAA


2020-02-01 price AAA 2.0 BBB
"""
display_scc_converted(ledger_simple_case_mult_acc)

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x2175813b3b0>

Note, that there are 2 unrealized gains postings for every price change. Each of these postings has different value of the **scc_bal_s_acc** meta. This allows to use beanquery to analyze which Balance Sheet accounts have caused unrealized gains.  

It is however possible to combine all postings to the unrealized gains account to a single posting, buy setting the **group_p_l_acc_tr** option to True.

In [10]:
display_scc_converted(ledger_simple_case_mult_acc, group_p_l_acc_tr = True)

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x21755f4f5c0>

Observe, that there is only one posting to the **Income:Unrealized-Gains:BBB-AAA** account, which makes the unrealized gains transaction more compact. As a downside the 2 posting-levels metadatas are not available now:
- scc_msg
- scc_bal_s_acc

Which means, that less information will be available for analysis

### <a id='toc1_2_5_'></a>[Unconvertable commodity](#toc0_)
Let us explore the situation, when we have a currency AAA, a target currency BBB and there is a third currency CCC, but there is no exchange rate between the BBB and the CCC. In another words the commodity CCC is what we call "an unconvertable in relation to the target currency", which is BBB in this case"

In [11]:
ledger_unconvertable_curr = """
option "operating_currency" "BBB" 

2020-01-01 open Assets:BankA
2020-01-01 open Assets:BankC
2020-01-01 open Equity:Opening-Balances
2020-01-01 open Expenses:Misc

2020-01-01 price AAA 1.0 BBB 

2020-01-01 * "Opening Balance"
    Assets:BankA                100 AAA
    Equity:Opening-Balances    -100 AAA
    Assets:BankC                100 CCC
    Equity:Opening-Balances    -100 CCC

2020-01-03 price AAA 1.5 BBB

2020-01-04 * "Buying something with AAA"
    Assets:BankA              -50 AAA
    Expenses:Misc              50 AAA
    
2020-01-04 * "Buying something with CCC"
    Assets:BankC              -50 CCC
    Expenses:Misc              50 CCC
    
    
2020-02-05 price AAA 2.0 BBB
"""

display_scc_converted(ledger_unconvertable_curr)

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x217582e0710>

Observe, that SCC has successfully handled such situation. The convertable commodity AAA is converted to the target currency BBB, the unconvertable commodity CCC remains unconvertable. The fact, that there are unconvertable commodities will not prevent a correct Net Worth Change analisys using the with beanquery, provided the following conditions are met:
 - unconvertable commodity shall remain unconvertable for the duration of the entire period being analyzed. In another words no **price** directive shall be introduced in the middle of the period being analyzed to provide an exchange rate between the previously  unconvertable commodity and a target commodity.
  
 - there should not be any conversions between unconvertable and convertable commodity withing the period being analyzed

#### <a id='toc1_2_5_1_'></a>[Usage of the "start_date" and "end_date" options](#toc0_)

Let us look at the previous example, but we will add a new **price** directive, which now provides an exchange rate information between the commodity CCC and BBB, but not at the very 1st date of our ledger. In another words the commodity CCC becomes convertable, but only somewhere in the middle of our financial history

In [12]:
ledger_unconvertable_curr_becomes_convertable = """

option "operating_currency" "BBB" 

2020-01-01 open Assets:BankA
2020-01-01 open Assets:BankC
2020-01-01 open Equity:Opening-Balances
2020-01-01 open Expenses:Misc

2020-01-01 price AAA 1.0 BBB 

2020-01-01 * "Opening Balance"
    Assets:BankA                100 AAA
    Equity:Opening-Balances    -100 AAA
    Assets:BankC                100 CCC
    Equity:Opening-Balances    -100 CCC

2020-01-03 price AAA 1.5 BBB

2020-01-04 price CCC 1.0 BBB ; <== now we can convert CCC to BBB, but only starting from this date

2020-01-04 * "Buying something with AAA"
    Assets:BankA              -50 AAA
    Expenses:Misc              50 AAA
    
2020-01-04 * "Buying something with CCC"
    Assets:BankC              -50 CCC
    Expenses:Misc              50 CCC
    
    
2020-02-05 price AAA 2.0 BBB
"""
display_scc_converted(ledger_unconvertable_curr_becomes_convertable)


An exception caught, while running the 'get_equiv_sing_curr_entries':
 
Problem is detected in provided beancount ledger.
There is a price entry for the commodity CCC vs BBB on the date 2020-01-04,
This commodity has been detected as unconvertable to the commodity BBB in the past.
Such situation is not allowed, as it would not allow to perform correct conversion of ledger to BBB


The SCC has raised an exception in this case. This is because the rule is violated, that whilst it is OK to have unconvertable commodity in the ledger, such unconvertable commodity shall remain unconvertable for the entire period being analyzed. Note, that this is not the limitation of the SCC tool, but a logical limitation of the data, provided to it. If this rule is not followed, it will not be possible to perform the Net Worth Change Analysis.  

To fix this situation one can either add a price directive with the date the same or earlier, than the 1st transaction or to run conversion for the reduced date range, by specifying the **start_date** parameter.

Let us specify the **start_date** parameter.

In [13]:
display_scc_converted(ledger_unconvertable_curr_becomes_convertable, start_date="2020-01-05")

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x21757ce1c70>

Observe, that the SCC has created the transaction *"Opening balance single currency transaction, equivalent to the Balance Sheet status on that date"*, at the date, previous to the specified **start_date**. This transaction compresses all previous transactions into a single one and converts all this to a single currency. Such ledger will still allow to do execute an unrealized gains analysis using beanquery in the reduced period from the **start_date**  until the end of the ledger.

Alternatively we can specify the **end_date**.

In [14]:
display_scc_converted(ledger_unconvertable_curr_becomes_convertable, end_date="2020-01-03")

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x217582e0110>

Note, that specifying the **end_date** makes the SCC to drop everything after it, thus still allowing to do the unrealized gain analysis in the period until including the **end_date**

### <a id='toc1_2_6_'></a>[Small value Balance error correction posting](#toc0_)

Sometimes (the exact scenario is still to be understood) the converted transaction does not balance with the very small value error.  Probably this is a part of the known beancount issue, which [has been discussed on the discussion group](https://groups.google.com/g/beancount/c/F6rc8QgTNEY/m/6X96jbslAAAJ). 
For now the **sing_curr_conv** detects such errors using the identical code as the original beancount does and then inserts a correction posting, which (for a lack of a better choice) is booked to the account **Income:Unrealized-Gains:\<Target Currency>-\<Target Currency>**

Example:

In [15]:
ledger_small_value_balance_error = """

option "operating_currency" "YYY"

2020-01-01 open Expenses:Misc1
2020-01-01 open Expenses:Misc2
2020-01-01 open Assets:Bank


2020-01-01 price YYY          92.9107970 XXX

2020-01-01 * "Buying something"
    Expenses:Misc1             3000.00 XXX
    Expenses:Misc2             16570.00 XXX
    Assets:Bank               -19570.00 XXX
"""
display_scc_converted(ledger_small_value_balance_error)

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x217577c0770>

One can see, that here a correction posting has to be booked

`Income:Unrealized-Gains:YYY-YYY    -0.0000000000000000000000001 YYY`